# TP2
### Problema 2

O Conway’s Game of Life é um exemplo bastante conhecido de um autómato celular. Neste problema vamos modificar as regras do autómato da seguinte forma:
- O espaço de estados é finito definido por uma grelha de células booleanas (morta=0/viva=1) de dimensão $\,N\times N\,$ (com $N>3$) identificadas por índices $\,(i,j)\in \{1..N\}$.  Estas $\;N^2\;$ células são aqui referidas como “normais”;
<br><br>
- Adicionalmente existem $\,2N+1\,$ “células da borda” que correspondem a um dos índices, $i$ ou $j$, ser zero. As células da borda têm valores constantes que, no estado inicial, são gerados aleatoriamente com uma probabilidade $\,\rho\,$ de estarem vivas;
<br><br>
- As células normais o autómato modificam o estado de acordo com a regra “B3/S23”: i.e. a célula nasce (passa de $0$ a $1$) se tem exatamente 3 vizinhos vivos e sobrevive (mantém-se viva) se o número de vizinhos vivos é 2 ou 3, caso contrário morre ou continua morta;

Pretende-se:

a. Construir uma máquina de estados finita que represente este autómato; são parâmetros do problema os parâmetros $\,N,\rho\,$ e a posição do  “centro”.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL, INT 
import random

### Inputs
São parâmetros do problema os parâmetros N, $\rho$ e a posição do centro.

A função `gera_quadrado` constroi o quadrado $\,3\times 3\,$ a partir do $centro$ que é passado como parâmetro.

In [2]:
def gera_quadrado(centro):
    return [(centro[0]+1, centro[1]), (centro[0], centro[1]+1),
           (centro[0]-1, centro[1]), (centro[0], centro[1]-1),
           (centro[0]+1, centro[1]+1), (centro[0]-1, centro[1]-1),
           (centro[0]+1, centro[1]-1), (centro[0]-1, centro[1]+1),
           (centro[0], centro[1])]

A função `declaracao` cria a i-ésima cópia das variáveis de estado, guardando-as num dicionário.

In [3]:
def declaracao(k, N):
    state = {}
    state['grelha'] = {}
    for i in range(N+1):
        for j in range(N+1):
            state['grelha'][i,j] = Symbol(f'{k}_grelha_{i}_{j}', INT)
            
    return state

A função `inicializacao` testa se um dado estado é um possível estado inicial do programa.

In [4]:
def inicializacao(state, N, quadrado, ro):
    expr = []
    
    #células normais
    for i in range(1, N+1):
        for j in range(1, N+1):
            #se (i,j) pertencer ao quadrado então essa posição da matriz é 1 senão 0
            if (i,j) not in quadrado:
                expr.append(Equals(state['grelha'][i,j], Int(0)))
            else:
                expr.append(Equals(state['grelha'][i,j], Int(1)))
    
    nums = [1,0]
    probs = [ro, 1-ro]
    expr.append(Equals(state['grelha'][0,0], Int(random.choices(nums, probs, k=1)[0])))
    
    #células de borda
    i = 0
    for j in range(1, N+1):
        expr.append(Equals(state['grelha'][i,j], Int(random.choices(nums, probs, k=1)[0])))
        
    j = 0
    for i in range(1, N+1):
        expr.append(Equals(state['grelha'][i,j], Int(random.choices(nums, probs, k=1)[0])))
            

    return And(elem for elem in expr)

A função de `transicao` impõe as condições necessárias para que seja possível transitar para o estado seguinte

In [40]:
def transicao(curr, prox, N):
    expr = []
    
    for i in range(N+1):
        for j in range(N+1):
            vizinhos = []
            
            for x in range(-1,2):
                for y in range(-1,2):
                    
                    if x != 0 or y != 0:
                        if i+x >= 0 and i+x < N+1 and j+y >= 0 and j+y < N+1:
                            vizinhos.append(curr['grelha'][i+x,j+y])
                            
            soma_igual_3 = Equals(Plus([elem for elem in vizinhos]), Int(3))
            celula_nasce = Equals(prox['grelha'][i,j], Int(1))
            soma_igual_2 = Equals(Plus([elem for elem in vizinhos]), Int(2))
            celula_mantem_se = Equals(prox['grelha'][i,j], curr['grelha'][i,j])
            celula_morre = Equals(prox['grelha'][i,j], Int(0))
            expr.append(Ite(soma_igual_3, celula_nasce, Ite(soma_igual_2, celula_mantem_se, celula_morre)))
    
    return And(expr)

A função `gera_traco` usa o SMT solver `z3` para um gerar possível traço de execução do programa de tamanho $k$.

In [25]:
def gera_traco(declaracao, inicializacao, transicao, k, N, quadrado, ro):

    with Solver(name = "z3") as solver:
        
        traco = [declaracao(i, N) for i in range(k)]
        
        solver.add_assertion(inicializacao(traco[0], N, quadrado, ro))
        
        for i in range(k-1):
            solver.add_assertion(transicao(traco[i], traco[i+1], N)) # condição ; # tem de ser verdadeira consoante o nosso estado atual e o seguinte
        
        
        if solver.solve():
            for i in range(k):
                print("Passo: ", i)
                for v in traco[i]:
                    for x in range(N+1):
                        for y in range(N+1):
                            print(solver.get_value(traco[i][v][x,y]), end = ' ')
                        print()
                print("-------------")

        else:
            print("Não encontrou solução")
        pass
    

b. Verificar se se conseguem provar as seguintes propriedades:<br><br>
$\quad$i. Todos os estados acessíveis contém pelo menos uma célula viva.<br>
$\quad$ii. Toda a célula normal está viva pelo menos uma vez em algum estado acessível.

A função `propriedade_1` testa se todos os estados acessíveis contém pelo menos uma célula viva.

In [26]:
def propriedade_1(state, N):
    formula = []
    
    for i in range(N+1):
        for j in range(N+1):
            formula.append(state['grelha'][i,j])
            
    return GE(Plus(formula), Int(1))

A função `propriedade_2` testa se toda a célula normal está viva pelo menos uma vez em algum estado acessível.

In [33]:
def propriedade_2(traco, N, K):
    formula = []
    
    for i in range(N+1):
        for j in range(N+1):
            soma = []
            for k in range(K):
                for v in traco[k]:
                    soma.append(traco[k][v][i,j])
            formula.append(GE(Plus(soma), Int(1)))
                
    return And(formula)

A função `testa_condicao1` usa o SMT solver `z3` para testar uma propriedade (passada como parâmetro) num traço de execução do programa.

In [45]:
def testa_condicao1(declaracao, inicializacao, transicao, propriedade, N, quadrado, ro, k):
    
    with Solver(name = "z3") as s:
        
        trace = [declaracao(i,N) for i in range(k+1)]
        
        s.add_assertion(inicializacao(trace[0], N, quadrado, ro))
        
        for i in range(k-1):
            s.add_assertion(transicao(trace[i], trace[i+1], N))
            s.add_assertion(propriedade(trace[i+1], N))
        
        if s.solve():
            print("A condição é válida nos", k, "passos.")
        else:
            print("Condição inválida")  
                
        

A função `testa_condicao2` usa o SMT solver `z3` para testar uma propriedade (passada como parâmetro) num traço de execução do programa.

In [34]:
def testa_condicao2(declaracao, inicializacao, transicao, propriedade, N, quadrado, ro, k):
    
    with Solver(name = "z3") as s:
        
        trace = [declaracao(i,N) for i in range(k+1)]
        
        s.add_assertion(inicializacao(trace[0], N, quadrado, ro))
        
        for i in range(k-1):
            s.add_assertion(transicao(trace[i], trace[i+1], N))
            
        s.add_assertion(propriedade_2(trace, N, k))
        
        if s.solve():
            print("A condição é válida nos", k, "passos.")
        else:
            print("Condição inválida")

### Exemplos

In [46]:
N = 5
centro = (3,3)
ro = 0.3
quadrado = gera_quadrado(centro)
gera_traco(declaracao,inicializacao,transicao,10,N,quadrado, ro)
testa_condicao1(declaracao, inicializacao, transicao, propriedade_1, N, quadrado, ro, 10)
testa_condicao2(declaracao, inicializacao, transicao, propriedade_2, N, quadrado, ro, 10)

Passo:  0
1 0 0 1 0 0 
0 0 0 0 0 0 
0 0 1 1 1 0 
1 0 1 1 1 0 
0 0 1 1 1 0 
0 0 0 0 0 0 
-------------
Passo:  1
0 0 0 0 0 0 
0 0 1 0 1 0 
0 1 1 0 1 0 
0 0 0 0 0 1 
0 1 1 0 1 0 
0 0 0 1 0 0 
-------------
Passo:  2
0 0 0 0 0 0 
0 1 1 0 0 0 
0 1 1 0 1 1 
0 0 0 0 1 1 
0 0 1 1 1 0 
0 0 1 1 0 0 
-------------
Passo:  3
0 0 0 0 0 0 
0 1 1 1 0 0 
0 1 1 0 1 1 
0 1 0 0 0 0 
0 0 1 0 0 1 
0 0 1 0 1 0 
-------------
Passo:  4
0 0 1 0 0 0 
0 1 0 1 1 0 
1 0 0 0 1 0 
0 1 0 1 1 1 
0 1 1 1 0 0 
0 0 0 1 0 0 
-------------
Passo:  5
0 0 1 1 0 0 
0 1 1 1 1 0 
1 1 0 0 0 0 
1 1 0 0 0 1 
0 1 0 0 0 0 
0 0 0 1 0 0 
-------------
Passo:  6
0 1 0 0 1 0 
1 0 0 0 1 0 
0 0 0 1 1 0 
0 0 1 0 0 0 
1 1 1 0 0 0 
0 0 0 0 0 0 
-------------
Passo:  7
0 0 0 0 0 0 
0 0 0 0 1 1 
0 0 0 1 1 0 
0 0 1 0 0 0 
0 1 1 0 0 0 
0 1 0 0 0 0 
-------------
Passo:  8
0 0 0 0 0 0 
0 0 0 1 1 1 
0 0 0 1 1 1 
0 1 1 0 0 0 
0 1 1 0 0 0 
0 1 1 0 0 0 
-------------
Passo:  9
0 0 0 0 1 0 
0 0 0 1 0 1 
0 0 0 0 0 1 
0 1 0 0 1 0 
1 0 0 1 0 0 
0 1 1 0

In [47]:
N = 10
centro = (7,7)
ro = 0.2
quadrado = gera_quadrado(centro)
gera_traco(declaracao,inicializacao,transicao,10,N,quadrado, ro)
testa_condicao1(declaracao, inicializacao, transicao, propriedade_1, N, quadrado, ro, 10)
testa_condicao2(declaracao, inicializacao, transicao, propriedade_2, N, quadrado, ro, 10)

Passo:  0
0 0 0 0 1 1 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  1
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 1 0 1 0 0 
0 0 0 0 0 1 0 0 0 1 0 
0 0 0 0 0 0 1 0 1 0 0 
0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  2
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 1 1 0 1 1 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  3
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 1 0 0 
0 0 0 0 0 1 0 0 0 1 0

In [48]:
N = 15
centro = (9,9)
ro = 0.1
quadrado = gera_quadrado(centro)
gera_traco(declaracao,inicializacao,transicao,20,N,quadrado, ro)
testa_condicao1(declaracao, inicializacao, transicao, propriedade_1, N, quadrado, ro, 20)
testa_condicao2(declaracao, inicializacao, transicao, propriedade_2, N, quadrado, ro, 10)

Passo:  0
0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  17
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
-------------
Passo:  18
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 